<a href="https://colab.research.google.com/github/Aashiff2003/DSGP-group-25/blob/Geethmi/yolov8_with_new_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics torch torchvision numpy matplotlib

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.9/112.6 GB disk)


In [ ]:
# Install and update Ultralytics and Ray Tune packages
!pip install -U ultralytics "ray[tune]"
display.clear_output()

In [ ]:
# Install W&B for logging
!pip install wandb
display.clear_output()

Data Cleaning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define dataset directories
# Train dir
train_image_dir = "/content/drive/MyDrive/DSGP/Model10/Data/train/images"
train_label_dir = "/content/drive/MyDrive/DSGP/Model10/Data/train/labels"

# Valid dir
valid_image_dir = "/content/drive/MyDrive/DSGP/Model10/Data/valid/images"
valid_label_dir = "/content/drive/MyDrive/DSGP/Model10/Data/valid/labels"

# Test dir
test_image_dir = "/content/drive/MyDrive/DSGP/Model10/Data/test/images"
test_label_dir = "/content/drive/MyDrive/DSGP/Model10/Data/test/labels"

In [ ]:
# Check image and label files count
import os

def count_files(dir):
  count=0
  for label_file in os.listdir(dir):
    count+=1
  print(f"{dir} - {count}")

count_files(train_image_dir)
count_files(valid_image_dir)
count_files(test_image_dir)
count_files(train_label_dir)
count_files(valid_label_dir)
count_files(test_label_dir)

/content/drive/MyDrive/DSGP/Model10/Data/train/images - 1490
/content/drive/MyDrive/DSGP/Model10/Data/valid/images - 200
/content/drive/MyDrive/DSGP/Model10/Data/test/images - 200
/content/drive/MyDrive/DSGP/Model10/Data/train/labels - 1490
/content/drive/MyDrive/DSGP/Model10/Data/valid/labels - 200
/content/drive/MyDrive/DSGP/Model10/Data/test/labels - 200


In [ ]:
# Check if there are segmenatation label files
def is_seg_file(filepath):
  file = open(filepath, "r")
  lines = file.readlines()
  for line in lines:
    no_of_values = line.strip().split()
    if len(no_of_values) > 5 :
      return True
  return False

# Remove lable file and image
def remove_files(image_dir,label_dir):
  for label_file in os.listdir(label_dir):
    label_path = os.path.join(label_dir, label_file)

    if label_file.endswith(".txt") and is_seg_file(label_path):
      # Find relavant image file
      image_file = label_file.replace(".txt", ".jpg")
      image_path = os.path.join(image_dir, image_file)

      # Delete label file and image
      os.remove(label_path)
      if os.path.exists(image_path):
        os.remove(image_path)

remove_files(train_image_dir,train_label_dir)
remove_files(valid_image_dir,valid_label_dir)
remove_files(test_image_dir,test_label_dir)

print("Dataset cleaned successfully!!")

Dataset cleaned successfully!!


In [ ]:
# Count after cleaning
count_files(train_image_dir)
count_files(valid_image_dir)
count_files(test_image_dir)
count_files(train_label_dir)
count_files(valid_label_dir)
count_files(test_label_dir)

/content/drive/MyDrive/DSGP/Model10/Data/train/images - 1485
/content/drive/MyDrive/DSGP/Model10/Data/valid/images - 199
/content/drive/MyDrive/DSGP/Model10/Data/test/images - 198
/content/drive/MyDrive/DSGP/Model10/Data/train/labels - 1485
/content/drive/MyDrive/DSGP/Model10/Data/valid/labels - 199
/content/drive/MyDrive/DSGP/Model10/Data/test/labels - 198


Hyperparameter Tunining with Tay Tune

In [ ]:
%cd /content/drive/MyDrive/DSGP/Model10

/content/drive/MyDrive/DSGP/Model10


In [ ]:
import wandb
wandb.login(key="a36b93a01adac06d58bd74c234e73d985dcfa963")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gimunr2003 (gimunr2003-robert-gordon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from ultralytics import YOLO
from ray import tune

# Load a YOLOv8s model
model = YOLO("yolov8n.pt")

search_space = {
    "lr0": tune.uniform(1e-5, 1e-1),
    "lrf": tune.uniform(0.01, 1.0),
    "momentum": tune.uniform(0.6, 0.98),
    "weight_decay": tune.uniform(0.0, 0.001),
    "warmup_epochs": tune.uniform(0.0, 5.0),
    "warmup_momentum":	tune.uniform(0.0, 0.95),
    "box":	tune.uniform(0.02, 0.2),
    "cls":	tune.uniform(0.2, 4.0),
    "mosaic":	tune.uniform(0.0, 1.0),
    "mixup":	tune.uniform(0.0, 1.0),
    "scale":	tune.uniform(0.0, 0.9),
    "translate":	tune.uniform(0.0, 0.9),
    "degrees":	tune.uniform(0.0, 45.0)
}

# Start tuning hyperparameters for YOLO11n training on the COCO8 dataset
result_grid = model.tune(
    data="/content/drive/MyDrive/DSGP/Model10/data.yaml",
    space=search_space,
    epochs=50,
    use_ray=True
)

💡 Learn about RayTune at https://docs.ultralytics.com/integrations/ray-tune


2025-03-18 22:39:06,199	INFO wandb.py:318 -- Already logged into W&B.


+--------------------------------------------------------------+
| Configuration for experiment     _tune_2025-03-18_22-39-06   |
+--------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator       |
| Scheduler                        AsyncHyperBandScheduler     |
| Number of trials                 10                          |
+--------------------------------------------------------------+

View detailed results here: /content/drive/MyDrive/DSGP/Model10/runs/detect/tune7/_tune_2025-03-18_22-39-06
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-03-18_22-28-25_186869_433/artifacts/2025-03-18_22-39-06/_tune_2025-03-18_22-39-06/driver_artifacts`

Trial status: 10 PENDING
Current time: 2025-03-18 22:39:06. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+-------------------------------------------------------------------------------------

(_WandbLoggingActor pid=12096) wandb: Currently logged in as: gimunr2003 (gimunr2003-robert-gordon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=12096) wandb: Tracking run with wandb version 0.19.8
(_WandbLoggingActor pid=12096) wandb: Run data is saved locally in /tmp/ray/session_2025-03-18_22-28-25_186869_433/artifacts/2025-03-18_22-39-06/_tune_2025-03-18_22-39-06/driver_artifacts/_tune_ccb36_00000_0_box=0.1126,cls=3.7321,degrees=3.9431,lr0=0.0808,lrf=0.9821,mixup=0.9439,momentum=0.8329,mosaic=0.8238,scale=0._2025-03-18_22-39-06/wandb/run-20250318_223917-ccb36_00000
(_WandbLoggingActor pid=12096) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=12096) wandb: Syncing run _tune_ccb36_00000
(_WandbLoggingActor pid=12096) wandb: ⭐️ View project at https://wandb.ai/gimunr2003-robert-gordon-university/YOLOv8-tune
(_WandbLoggingActor pid=12096) wandb: 🚀 View run at https://wandb.ai/gimunr2003-robert-gordon-un

(_tune pid=11987) Ultralytics 8.3.92 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
(_tune pid=11987) engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/DSGP/Model10/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop

...

(_tune pid=15108) optimizer: 'optimizer=auto' found, ignoring 'lr0=0.052474092845456274' and 'momentum=0.6240371715871752' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
(_tune pid=15108) optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005777357909657053), 63 bias(decay=0.0)
Trial status: 2 RUNNING | 1 ERROR | 7 PENDING
Current time: 2025-03-18 22:43:15. Total running time: 4min 9s
Logical resource usage: 2.0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name          status            lr0        lrf     momentum     weight_decay     warmup_epochs     warmup_momentum         box        cls     mosaic      mixup      scale     translate     degrees |
+-------------------------------------------

  0%|          | 0/93 [00:00<?, ?it/s]


(_tune pid=15108) TensorBoard: model graph visualization added ✅
(_tune pid=15108) Image sizes 640 train, 640 val
(_tune pid=15108) Using 0 dataloader workers
(_tune pid=15108) Logging results to runs/detect/train
(_tune pid=15108) Starting training for 50 epochs...
(_tune pid=15108) 
(_tune pid=15108)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
(raylet) WARNING: 14 PYTHON worker processes have been started on node: f822ac4f6fc6af06e4154e76f873183e0755e2ac08c558d87e2739a4 with address: 172.28.0.12. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
Trial status: 2 RUNNING | 1 ERROR | 7 PENDING
Current time: 2025-03-18 22:43:45. Total running time: 4min 39s
Logical resource usage: 2.0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+--------------------------------------------------------------------------------------

2025-03-18 22:43:46,440	WARNING util.py:201 -- The `on_step_begin` operation took 0.712 s, which may be a performance bottleneck.
(_WandbLoggingActor pid=15956) wandb: Currently logged in as: gimunr2003 (gimunr2003-robert-gordon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
(raylet) [2025-03-18 22:44:27,078 E 6619 6619] (raylet) node_manager.cc:3219: 23 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: f822ac4f6fc6af06e4154e76f873183e0755e2ac08c558d87e2739a4, IP: 172.28.0.12) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.28.0.12`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the

RuntimeError: Caught unexpected exception: Task was killed due to the node running low on memory.
Memory on the node (IP: 172.28.0.12, ID: f822ac4f6fc6af06e4154e76f873183e0755e2ac08c558d87e2739a4) where the task (actor ID: d3b9abcab82eb2fee7f4d6e501000000, name=ImplicitFunc.__init__, pid=15108, memory used=3.72GB) was running was 12.11GB / 12.67GB (0.955683), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: bcc85184827da8786f7e5ee04fd7d75d6c0ea1dd027310db4d0b504f) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 172.28.0.12`. To see the logs of the worker, use `ray logs worker-bcc85184827da8786f7e5ee04fd7d75d6c0ea1dd027310db4d0b504f*out -ip 172.28.0.12. Top 10 memory users:
PID	MEM(GB)	COMMAND
11987	4.43	ray::ImplicitFunc.train
15108	3.72	ray::ImplicitFunc.train
433	0.54	/usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-608193b8-c3ef...

Train the model


In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8s.pt")

# Train the model
results = model.train(
    data="data.yaml",
    epochs=100,
    imgsz=640,
    plots=True,
    batch=16,
    pretrained=True,
    augment=True,
)

print(results)
# !yolo train data=your_dataset.yaml model=yolov8s.pt imgsz=640 augment=true

Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_wid

100%|██████████| 755k/755k [00:00<00:00, 18.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

train: Scanning /content/drive/MyDrive/DSGP/Model10/Data/train/labels.cache... 1485 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1485/1485 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

    100/100      7.57G       1.47     0.7375      1.121         72        640: 100%|██████████| 93/93 [00:32<00:00,  2.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.95it/s]

                   all        199       1826      0.743      0.742      0.767      0.384



100 epochs completed in 1.102 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 22.5MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:07<00:00,  1.09s/it]


                   all        199       1826      0.692      0.722      0.752      0.381
Speed: 0.4ms preprocess, 15.8ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/train4
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ee1fb9df1d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.03

Evaluation

In [ ]:
# Load the trained YOLOv8 model
model = YOLO("/content/drive/MyDrive/DSGP/Model10/runs/detect/train4/weights/best.pt")

# Perform validation
results = model.val(data="data.yaml", imgsz=640, batch=16)

Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /content/drive/MyDrive/DSGP/Model10/Data/valid/labels.cache... 199 images, 0 backgrounds, 0 corrupt: 100%|██████████| 199/199 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:05<00:00,  2.36it/s]


                   all        199       1826      0.756      0.733      0.772      0.388
Speed: 2.2ms preprocess, 9.6ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/val4


In [ ]:
import os

# Load the trained YOLOv8 model
model = YOLO("/content/drive/MyDrive/DSGP/Model10/runs/detect/train4/weights/best.pt")

# Path to your test dataset folder
test_dataset_path = "/content/drive/MyDrive/DSGP/Model10/Data/test/images"

# Confidence threshold
conf_threshold = 0.5

# Get all image file paths
image_files = [f for f in os.listdir(test_dataset_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Dictionary to store results
detection_counts = {}

# Run YOLOv8 on each image in the test dataset
for image_file in image_files:
    image_path = os.path.join(test_dataset_path, image_file)
    results = model(image_path, conf=conf_threshold)  # Perform inference

    # Get the number of objects detected
    num_objects = len(results[0].boxes)

    # Store the result
    detection_counts[image_file] = num_objects

# Print results
# for image, count in detection_counts.items():
#     print(f"{image}: {count} objects detected")



image 1/1 /content/drive/MyDrive/DSGP/Model10/Data/test/images/-E3-83-80-E3-82-A6-E3-83-B3-E3-83-AD-E3-83-BC-E3-83-89-20-18-_jpeg.rf.93da798ad1d99e489f29fe176a5c53f6.jpg: 640x640 1 Bird, 15.9ms
Speed: 2.3ms preprocess, 15.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/DSGP/Model10/Data/test/images/-E3-83-80-E3-82-A6-E3-83-B3-E3-83-AD-E3-83-BC-E3-83-89-20-4-_jpeg.rf.c51c0e7ea6cab6dcc438ac62fe5ccb8f.jpg: 640x640 1 Bird, 15.9ms
Speed: 2.3ms preprocess, 15.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/DSGP/Model10/Data/test/images/000098_jpg.rf.b4c3979339b674a66e5654cc4aedd671.jpg: 640x640 9 Birds, 15.8ms
Speed: 2.2ms preprocess, 15.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/DSGP/Model10/Data/test/images/000114_jpg.rf.2f107e1eddfedce8d5a70a5e65331083.jpg: 640x640 10 Birds, 15.8ms
Speed: 1.6ms preprocess, 15.8ms inferen